In [62]:
import os 
import pandas as pd
import numpy as np
import glob

## Loading the samplesheet data

In [63]:
# load the sample sheet with the number of loops
wc = pd.read_csv('../samplesheets/samplesheet.fithichip_hp.wc.txt',
                 sep='\t',
                 names=['sample', 'num_loops'],
                 header=None)

# extract metadata from the file names
def get_meta(x):
    meta = x.split('/')
    bn = meta[-1]
    bn_split = bn.split('.')
    res = bn_split[5]
    ref = meta[2]
    loop_source = meta[4]
    peak_source = meta[5]
    peak_caller = meta[6]
    stringency = meta[7]
    loop_caller = 'fithichip' if 'FitHiC' in meta[8] else 'other'
    return((ref, res, loop_source, peak_source, peak_caller, stringency, loop_caller))
metadata = wc['sample'].apply(get_meta).values.tolist()
metadata = list(zip(*metadata))
refs, res, loop_sources, peak_sources, peak_callers, stringencies, loop_callers = metadata

wc['ref'] = refs
wc['res'] = res
wc['loop_source'] = loop_sources
wc['peak_source'] = peak_sources
wc['peak_caller'] = peak_callers
wc['stringency'] = stringencies
wc['loop_caller'] = loop_callers

## Sample-level Summary For Loop Counts


In [64]:
# count the number of samples with zero and non-zero number of loops
grps = wc.groupby(['ref', 'res', 'stringency', 'peak_caller', 'loop_caller'])

# perform the value count across groups
value_counts = grps.num_loops.value_counts(bins=[-np.inf, 0, np.inf]).to_frame()
value_counts.columns = ['count']
value_counts.reset_index(inplace=True)
value_counts = value_counts.pivot(index=['ref', 'res', 'stringency', 'peak_caller', 'loop_caller'],
                   columns=['num_loops'],
                   values=['count'])
value_counts.columns = value_counts.columns.set_levels(['zero', 'non-zero'], level=1)

# add a total column
value_counts[('count', 'total')] = value_counts.sum(axis=1)

In [65]:
value_counts

count               
num_loops                                            zero non-zero total
ref  res   stringency peak_caller       loop_caller                     
hg38 10000 loose      fithichip-utility fithichip      23      455   478
           stringent  fithichip-utility fithichip      56      422   478
     25000 loose      fithichip-utility fithichip      14      464   478
           stringent  fithichip-utility fithichip      44      434   478
     5000  loose      fithichip-utility fithichip      34      444   478
           stringent  fithichip-utility fithichip      81      397   478

In [66]:
value_counts.sum(axis=0).to_frame().T

count               
num_loops  zero non-zero total
0           252     2616  2868

The most important number will be non-zero, we will use this number to ensure that we correctly
producing the correct number of visualization files.

Count the existence of loops in the sample sheet

In [76]:
# count the main interactions files
glob_str = '../../../../'
glob_str += 'results/shortcuts/hg38/loops/hichip/hichip/fithichip-utility/*/*.*.interactions_FitHiC_Q0.01.bed'
num_main = len(glob.glob(glob_str.format(stringency='*')))

# count the biginteractions files
glob_str = '../../../../'
glob_str += 'results/shortcuts/hg38/loops/hichip/hichip/fithichip-utility/*/*.*.interaction.bb'
biginter_list = glob.glob(glob_str.format(stringency='*'))
num_biginter = len(biginter_list)

# count the longrange files
glob_str = '../../../../'
glob_str += 'results/shortcuts/hg38/loops/hichip/hichip/fithichip-utility/*/*.*.longrange.bed.gz'
longrange_list = glob.glob(glob_str.format(stringency='*'))
num_longrange = len(longrange_list)

# print the counts
print(num_main, num_biginter, num_longrange)

2868 2616 2485


## Further investigate problematic samples

In [8]:
wc['bb_file'] = wc['sample'].replace('bed$', 'interaction.bb', regex=True)

# check if the file exists
def check_file(x, root_dir='../../../../'):
    fn = os.path.join(root_dir, x)
    return(os.path.exists(fn))
wc['bb_file_exists'] = wc['bb_file'].apply(check_file)

In [9]:
nonzero_data = wc.loc[(wc.num_loops > 0)]

missing_data = wc.loc[(wc.num_loops > 0) & (wc.biginter_exists == False)]
missing_arrayids = missing_data.index.values + 1

print(missing_arrayids)
missing_arrayids = missing_arrayids.astype(str)
rerun_ids = ','.join(missing_arrayids)

print(rerun_ids)

# delete accidentally created files

In [44]:
zeros = wc.loc[(wc.num_loops == 0)]

In [60]:
zeros.shape


(252, 9)

In [48]:
import re

In [49]:
'bed$', 'interaction.bb', 'test.bed')

'test.interaction.bb'

In [75]:
# i = 0
# for f in zeros['sample']:
#     f = '../../../../' + f
#     lr = re.sub('bed$', 'longrange.bed.gz', f)
#     idx = re.sub('bed$', 'longrange.bed.gz.tbi', f)
#     if os.path.exists(lr):
#         print(os.path.exists(lr), os.path.exists(idx))
#         i += 1
#         os.remove(lr)
#         os.remove(idx)

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [74]:
i

184